In [13]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm, trange


In [14]:
data = pd.read_csv('influence_data.csv')

In [15]:
data

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990
...,...,...,...,...,...,...,...,...
42765,580300,Sufjan Stevens,Pop/Rock,1990,3661738,Rosemary & Garlic,Pop/Rock,2010
42766,261309,Vybz Kartel,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010
42767,467203,Michael Jackson,R&B;,1960,3670556,Trinidad Cardona,R&B;,2010
42768,2518003,Popcaan,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010


In [16]:
# Set up a DataFrame to look up the genre and active_start_decade of the musicians
influencer_info = data.iloc[:,[0,2,3]].set_index('influencer_id')

influencer_info =influencer_info.T.rename(
    index={'influencer_main_genre':'genre','influencer_active_start':'decade'}).T
follower_info = data.iloc[:,[4,6,7]].set_index('follower_id')

follower_info = follower_info.T.rename(
    index={'follower_main_genre':'genre','follower_active_start':'decade'}).T
musician_info=pd.concat([influencer_info,follower_info])
musician_info = musician_info.groupby(musician_info.index).first()

In [17]:
musician=list(musician_info.index)

In [18]:
len(musician)

5603

In [6]:
G=nx.DiGraph()
for index, row in data.iterrows():
    G.add_edge(row.influencer_id, row.follower_id)


In [10]:
distance=pd.DataFrame(index=musician,columns=musician,dtype=float).fillna(value = 0)
for i in musician:
    for j in musician:
        try:
            distance.loc[i,j]=nx.shortest_path_length(G,source=i, target=j)
        except :
            distance.loc[i,j]=np.inf



In [36]:
mean_distance=pd.DataFrame(index=musician,columns=musician,dtype=float).fillna(value = 0)
for i in musician:
    if i%100==0:
        print(i)
    for j in musician:
#         divi=1/(np.inf if distance.loc[i,j] == 0 else distance.loc[i,j])+1/(np.inf if distance.loc[j,i] == 0 else distance.loc[j,i])
#         if divi != 0:
        mean_distance.loc[i,j]=2/((1/distance.loc[i,j])+(1/distance.loc[j,i]))
#         else:
#             mean_distance.loc[i,j] = np.inf         
                                   
for i in musician:
    mean_distance.loc[i,i]=np.nan

<ipython-input-36-d3a14d252691>:8: RuntimeWarning: divide by zero encountered in double_scalars
  mean_distance.loc[i,j]=2/((1/distance.loc[i,j])+(1/distance.loc[j,i]))


29600
38100
52900
66800
67900
70800
75100
97200
103600
134600
147200
182900
196200
212200
233300
258000
259800
268700
282800
314600
331400
363100
407300
408500
450700
482600
515600
569600
573100
580300
662800
674600
675500
688400
692100
747200
762500
765900
785500
799100
800100
809900
816900
817000
838300
840000
861600
896600
918400
918600
922200
944700
955700
1528100
2942900
3376900


In [40]:
distance_list = 1/np.array(mean_distance)

train_x_list=distance_list.tolist()


In [42]:
mean_distance.to_csv('mean_distance.csv')